In [112]:
import os
import numpy as np
from matplotlib import pyplot as plot
import math
from math import pi as pi
from astropy import units as u
import pandas as pd

In [258]:
#input variables - roughly based on those used in Lowell manuel
#example values for some of these variables can be found in the PDFs in the repository
# to do: input known values into tables (i.e. lunar brightness, airmass, etc)

SNRi = input('SNR: ')
SNR = float(SNRi)
mag_rawi = input('Magnitude of target star: ')
mag_raw = float(mag_rawi)
exp_timei = input('Exposure time: ')
if len(exp_timei)==0: # len function to find the length of list
    exp_time = 0
else:
    exp_time = float(exp_timei)
        
    
    
#Just keep these values for now
bandpass = input('Bandpass: ') #possible values: UVBIR. Currently testing V
seeing = 2
binning = 2
lunar_age = 3 #example value, for V band, lunar age of 3 means 21.7 lunar brightness
airmass = 1
mag_norm = 20 #20 is what lowell uses


#possible other values:
# psf = [0]
# lunar_bright = 21.7 
# signal_norm = 100  #based on Lowell manual for V band


SNR: 10
Magnitude of target star: 12.5
Exposure time: 
Bandpass: B


In [259]:
# data into dataframe
# NOTE: extinction values need to be edited

data = {'U':[100,.6,[22.0,21.5,19.9,18.5,17.0]],'B':[730,0.4,[22.7,22.4,21.6,20.7,19.5]],'V':[680,0.156,[21.8,21.7,21.4,20.7,20.0]],'R':[670,0.16656,[20.9,20.8,20.6,20.3,19.9]],'I':[450,0.099,[19.9,19.9,19.7,19.5,19.2]]
}
inputvalues = ['signal_norm_val','extinct_val','lunar_val']

# bandpass[0] = [100,0.352,[22.0,21.5,19.9,18.5,17.0]]
# bandpass[1] = [730,0.142,[22.7,22.4,21.6,20.7,19.5]]
# bandpass[2] = [680,0.156,[21.8,21.7,21.4,20.7,20.0]]
# bandpass[3] = [670,0.16656,[20.9,20.8,20.6,20.3,19.9]]
# bandpass[4] = [450,0.099,[19.9,19.9,19.7,19.5,19.2]]

df = pd.DataFrame(data, index = inputvalues)


In [260]:
#set values for the following calculations based on the user input:

signal_norm = df.loc['signal_norm_val',bandpass]
extinct = df.loc['extinct_val',bandpass]

if lunar_age == 0:
    lunar_index = 0
if lunar_age == 3:
    lunar_index = 1
if lunar_age == 7:
    lunar_index = 2
if lunar_age == 10:
    lunar_index = 3
if lunar_age == 14:
    lunar_index = 4
    
lunar_bright = df.loc['lunar_val',bandpass][lunar_index]


print("signal norm: ",signal_norm)
print("extinct: ",extinct)
print("lunar_bright: ",lunar_bright)


signal norm:  730
extinct:  0.4
lunar_bright:  22.4


In [261]:
#calculate pixels
scale = .12

#if using seeing:
px = (pi*(.6676*seeing)**2)/((scale*binning)**2) 
print(px)


97.23442193029152


In [262]:
#calculate sky noise

N_sky_split = signal_norm * 10**((-(lunar_bright - mag_norm))/2.5)
area = scale**2 * binning**2
total_area = area * px
N_sky_wtotalarea = total_area * N_sky_split

#this is the N_sky we are working with RIGHT NOW, in units e/sec/px: this is W/O total area
N_sky = signal_norm * 10**((-(lunar_bright - mag_norm))/2.5) * scale**2 * binning**2

print(N_sky_split)
print(N_sky_wtotalarea)
print(N_sky)
print("This is the area:", area)



80.04290831845262
448.29653298993
4.6104715191428705
This is the area: 0.0576


In [263]:
#calculate signal from magnitude, with correction

mag_corr = (mag_raw + extinct*airmass)
signal = signal_norm * 10**((-(mag_corr - mag_norm))/2.5) 

print(mag_corr)
print(signal)
print(mag_raw)



12.9
505036.60877082346
12.5


In [264]:
#calculate SNR or exposure time or magnitude
#set Nron = 6, as seen in DTC/LMI manual (crossed checked with calculation with output of Lowell ETC)
#ignore dark current

N_ron = 6

#SNR calculation:
def ETC_lowell_test(SNR, exp_time, mag_raw, signal, signal_norm, px, N_sky, N_ron, airmass, extinct):
    response = [SNR, exp_time, 0]
    if SNR == 0:
        SNR = (signal * exp_time)/(math.sqrt(signal*exp_time + N_sky*exp_time*px + (N_ron**2)*px))
        mag_corr_output = mag_raw + extinct*airmass


    if exp_time == 0:
        a = signal**2
        b = -(SNR**2) * (signal + (px*N_sky))
        c = -(SNR**2) * px * N_ron**2
        exp_time = (-b +(math.sqrt((b**2) - (4*a*c))))/(2*a)
        mag_corr_output = mag_raw + extinct*airmass

    if mag_raw == 0:
        a = exp_time**2
        b = -exp_time * SNR**2
        c = (-SNR**2 * N_sky * px * exp_time) + (px * N_ron**2)
        signal = (-b + math.sqrt((b**2 - 4)*a*c))/(2*a)
        mag_raw = 2.5 * math.log((signal/signal_norm),10) + mag_norm
        mag_corr_output = mag_raw - (extinct*airmass)
    
    response = [SNR, exp_time, mag_corr_output]
    return response



In [265]:
#print result
#print(SNR, "\n", exp_time, "\n",mag_raw, "\n",signal, "\n",signal_norm, "\n",px,"\n", N_sky, "\n",N_ron, "\n",airmass, "\n",extinct)

answer = ETC_lowell_test(SNR, exp_time, mag_raw, signal, signal_norm, px, N_sky, N_ron, airmass, extinct)
print("This is the SNR:", answer[0], "\nThis is the exposure time:", answer[1], "\nThis is the corrected magnitude:", answer[2] )

#This is a value that is known after all 3 values (mag, time and SNR) are calculated)
N_sky_result = math.sqrt(total_area * N_sky_split *  answer[1])
print("Noise from sky:",N_sky_result)
print("Noise from CCD:",math.sqrt(px * N_ron**2))
print("Noise from signal:", math.sqrt(signal*answer[1]))
print("This is the count from the star:", signal * answer[1])

#SNR evaluation
if SNR <= 3:
    print("You have low SNR, cannot use this data")
if SNR < 10 and SNR >= 5:
    print("There's an object out there!...somewhere")
if SNR >= 10 and SNR < 90:
    print("You can use this data for measurement")
if SNR >= 90 and SNR <= 100:
    print("Your SNR is amazing!")
    

This is the SNR: 10.0 
This is the exposure time: 0.0012747634530367532 
This is the corrected magnitude: 12.9
Noise from sky: 0.7559576948339423
Noise from CCD: 59.164509543226124
Noise from signal: 25.37325779845124
This is the count from the star: 643.8022113066668
You can use this data for measurement


In [255]:
        a = 5.75**2
        b = -(SNR**2) * (5.75 + (px*1.44684658455))
        c = -(SNR**2) * px * N_ron**2
        exp_time = (-b +(math.sqrt((b**2) - (4*a*c))))/(2*a)
        mag_corr_output = mag_raw + extinct*airmass
        
        print(exp_time)
        

465.6358065693187
